# Data preparation

Copyright (c) 2017 Alexander Menkin

Use of this source code is governed by an MIT-style license that can be found in the LICENSE file at
https://github.com/miloiloloo/diploma_2017_method/blob/master/LICENSE

In [ ]:
import numpy as np
import pickle

In [ ]:
def load_processed_depth_data(filepath):
    try:
        f = open(filepath, 'rb')
        save = pickle.load(f)
        number_of_frames = save['number_of_frames']
        assembly_offsets = save['assembly_offsets']
        assemblies = save['assemblies']
        indecies = save['indecies'] 
        f.close()
        del save
    except Exception as e:
        raise
    return (number_of_frames, assembly_offsets, assemblies, indecies)

In [ ]:
def load_processed_laboras_data(filepath):
    try:
        f = open(filepath, 'rb')
        save = pickle.load(f)
        time_intervals = save['time_intervals']
        probabilities = save['probabilities']
        f.close()
        del save
    except Exception as e:
        raise
    return (time_intervals, probabilities)

In [ ]:
def write_data_for_learning(file_path, assemblies, probabilities):
    try:
        f = open(file_path, 'wb')
        save = {
            'assemblies': assemblies,
            'probabilities': probabilities,
        }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
        f.close()
    except Exception as e:
        print('Unable to save data to', file_path, ':', e)
        raise
    print('Data has been saved')
    return

In [ ]:
def get_learning_data(start_time, stop_time,
                      time_intervals, probabilities,
                      number_of_frames,
                      indecies, assembly_offsets, assemblies):
    ''' Combine processed laboras data and assemblies data from depth sensor, where:
        It returns a tuple (data, probabilities)
        Remember: all time is in seconds!!!
    '''
    
    ''' CHECK INPUT '''
    ''' Incorrect input 1: start time more then stop time '''
    if start_time > stop_time:
        raise Exception
    ''' Incorrect input 2: incorrect time intervals '''
    for time_interval_idx in range(0, time_intervals.shape[0]):
        if time_interval_idx > 0:
            if time_intervals[time_interval_idx, 0] < time_intervals[time_interval_idx - 1, 1]:
                raise Exception
        if time_intervals[time_interval_idx, 0] > time_intervals[time_interval_idx, 1]:
            raise Exception
    ''' Incorrect input 3: incorrect probabilities '''
    if np.min(probabilities) < 0:
        raise Exception
    ''' Incorrect input 4: number of indecies is not equal number of assemblies '''
    if indecies.shape[0] != assemblies.shape[0]:
        raise Exception
    ''' Incorrect input 5: incorrect combination of number of frames, indecies and assembly_offsets '''
    
    
    ''' ALGORITHM '''
    ''' Predefined undef class index and undef probability limit'''
    undef_class_idx = 0
    undef_probability_limit = 0.5
    
    ''' Duration of the experiment '''
    duration = stop_time - start_time
    
    ''' Time per frame under conditions:
        1. Frist frame (0) is matched to the start_time;
        2. Last frame (number_of_frames - 1) is matched to the stop_time.
    ''' 
    time_per_frame = float(duration) / (number_of_frames - 1)
        
    ''' Containers for learning data '''
    learning_assemblies = np.ndarray(shape=assemblies.shape, dtype=assemblies.dtype)
    learning_probabilities = np.ndarray(shape=(assemblies.shape[0], probabilities.shape[1]), dtype=probabilities.dtype)
    learning_data_counter = 0
    
    ''' ALGORITHM '''
    for assembly_idx in range(0, assemblies.shape[0]):
        
        ''' Prepare data for assembly '''
        is_assembly_acceptable = True
        assembly_probabilities = np.zeros(shape=(probabilities.shape[1]), dtype=probabilities.dtype)
        patch_frame_indecies = assembly_offsets + indecies[assembly_idx]
        
        ''' Count assembly_probabilities '''
        for patch_frame_idx in patch_frame_indecies:
            
            ''' Time for patch's frame '''
            time_of_frame = start_time + time_per_frame*patch_frame_idx
            
            ''' Find time interval '''
            time_interval_has_been_found = False
            time_interval_for_patch_idx = 0
            for time_interval_idx in range(0, time_intervals.shape[0]):
                if time_of_frame >= time_intervals[time_interval_idx, 0] and time_of_frame < time_intervals[time_interval_idx, 1]:
                    time_interval_for_patch_idx = time_interval_idx
                    time_interval_has_been_found = True
                    break
                if time_interval_idx == time_intervals.shape[0] - 1 and round(time_of_frame) == time_intervals[time_interval_idx, 1]:
                    time_interval_for_patch_idx = time_interval_idx
                    time_interval_has_been_found = True
                    break
            
            if time_interval_has_been_found:    
                ''' Time interval has been found '''
                if probabilities[time_interval_for_patch_idx, undef_class_idx] <= undef_probability_limit:
                    ''' Probabilities of this time interval are enough good '''
                    ''' Add probabilities of this patch's frame to assembly probabilities '''
                    assembly_probabilities = assembly_probabilities + probabilities[time_interval_for_patch_idx, :]
                    continue
            
            ''' Time interval has not been found or probabilities are not good '''
            ''' Skip this assembly '''
            is_assembly_acceptable = False
            break
        
        ''' if sum of assembly's probabilities are less or equal 0 then they are unacceptable '''
        if np.sum(assembly_probabilities) <= 0:
            is_assembly_acceptable = False
        
        if is_assembly_acceptable:
                ''' Normalize assembly probabilities '''
                assembly_probabilities = assembly_probabilities / np.sum(assembly_probabilities)
                ''' Add to learning data '''
                learning_assemblies[learning_data_counter, :, :, :] = assemblies[assembly_idx, :, :, :]
                learning_probabilities[learning_data_counter, :] = assembly_probabilities
                learning_data_counter = learning_data_counter + 1
                        
        ''' End of cycle '''
    
    return (learning_assemblies[0 : learning_data_counter, :, :, :], learning_probabilities[0 : learning_data_counter, :])

In [ ]:
''' Depth parameters '''
depth_data_size = 64
# Warning: do not forger to set parameter
depth_data_parameters = [
    [1, 1],
    [1, 2],
    [2, 1],
    [1, 4],
    [2, 2],
    [4, 1],
    [1, 8],
    [2, 4],
    [4, 2],
    [8, 1]
]

In [ ]:
# Warning: do not forger to set parameter
depth_data_parameters_index = 9
depth_data_offset = depth_data_parameters[depth_data_parameters_index][0]
depth_data_neighbours_per_side = depth_data_parameters[depth_data_parameters_index][1]

input_processed_depth_data_directory_path = "./processed_depth_data/size_" + str(depth_data_size) + "_offset_" + str(depth_data_offset) + "_left_" + str(depth_data_neighbours_per_side) + "_right_" + str(depth_data_neighbours_per_side) + "/"
print(input_processed_depth_data_directory_path)

output_processed_depth_data_directory_path = "./learning_data/size_" + str(depth_data_size) + "_offset_" + str(depth_data_offset) + "_left_" + str(depth_data_neighbours_per_side) + "_right_" + str(depth_data_neighbours_per_side) + "/"
print(output_processed_depth_data_directory_path)

In [ ]:
# Warning: do not forger to set parameter
experiment_idx = 4

# Warning: do not forger to write dir path
input_processed_laboras_file_directory_path = "" + str(experiment_idx) + ".pickle" 
print(input_processed_laboras_file_directory_path)

''' Load processed laboras data '''
time_intervals, probabilities = load_processed_laboras_data(input_processed_laboras_file_directory_path)
start_time = np.min(time_intervals[:,0])
stop_time = np.max(time_intervals[:,1])

In [ ]:
# Warning: do not forger to set parameter
part_idx = 4

name = str(experiment_idx) + "_" + str(part_idx)

input_processed_depth_data_file_path = input_processed_depth_data_directory_path + name + ".pickle"
print(input_processed_depth_data_file_path)

output_learning_data_file_path = output_processed_depth_data_directory_path + name + ".pickle"
print(output_learning_data_file_path)

In [ ]:
''' Load processed depth data '''
number_of_frames, assembly_offsets, assemblies, indecies = load_processed_depth_data(input_processed_depth_data_file_path)

if experiment_idx == 2:
    for indecies_idx in range(0, indecies.shape[0]):
        if indecies[indecies_idx] > number_of_frames:
            indecies = indecies[0:indecies_idx]
            assemblies = assemblies[0:indecies_idx, :, :, :]
            break
                
if indecies.shape[0] != 0 and assemblies.shape[0] != 0:
    ''' Get learning data '''
    experiment_learning_assemblies, experiment_learning_probabilities = get_learning_data(start_time,
                                                                                          stop_time,
                                                                                          time_intervals,
                                                                                          probabilities,
                                                                                          number_of_frames,
                                                                                          indecies,
                                                                                          assembly_offsets,
                                                                                          assemblies)
    write_data_for_learning(output_learning_data_file_path, experiment_learning_assemblies, experiment_learning_probabilities)